# Import des modules

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error, r2_score
import re
from datetime import datetime


(On s'attend à ce que le preprocessing soit fait conformément au reste du notebook et que le réultat soit enregistré dans une dataframe `df`)

# Optimisation des hyperparamètres de Prophet
Pour ne pas faire d'optimisation, il suffit de mettre `epochs=0`.

In [ ]:
def prophet_hyperparameter_tuning(df, epochs=100, learning_rate=0.01):
    # Initialisation aléatoire des paramètres
    current_cps = np.random.uniform(0.001, 0.5)
    current_sps = np.random.uniform(1, 20)
    best_loss = float('inf')
    best_params = {}
    history = []
    
    for epoch in range(epochs):
        # Entraînement avec les paramètres courants
        model = Prophet(
            changepoint_prior_scale=current_cps,
            seasonality_prior_scale=current_sps,
            yearly_seasonality=True
        )
        current_cps = np.clip(current_cps, 0.001, 0.5)
        current_sps = np.clip(current_sps, 1, 20)
        model.fit(df)
        
        # Validation croisée (plus robuste qu'un simple train-test split)
        df_cv = cross_validation(model, initial='365.25 days', period='180 days', horizon='90 days')
        df_p = performance_metrics(df_cv)
        current_loss = np.mean(df_p['rmse'])
        
        # Mise à jour des meilleurs paramètres
        if current_loss < best_loss:
            best_loss = current_loss
            best_params = {
                'changepoint_prior_scale': current_cps,
                'seasonality_prior_scale': current_sps
            }
        
        # Calcul des gradients (approximation numérique)
        epsilon = 1e-5
        delta = 0.01
        
        # Gradient pour changepoint_prior_scale
        model_cps_plus = Prophet(
            changepoint_prior_scale=current_cps + epsilon,
            seasonality_prior_scale=current_sps
        )
        current_cps = np.clip(current_cps, 0.001, 0.5)
        current_sps = np.clip(current_sps, 1, 20)
        model_cps_plus.fit(df)
        loss_cps_plus = np.mean(performance_metrics(cross_validation(
            model_cps_plus, initial='365.25 days', period='180 days', horizon='90 days'
        ))['rmse'])
        
        grad_cps = (loss_cps_plus - current_loss) / epsilon
        
        # Gradient pour seasonality_prior_scale
        model_sps_plus = Prophet(
            changepoint_prior_scale=current_cps,
            seasonality_prior_scale=current_sps + epsilon
        )
        current_cps = np.clip(current_cps, 0.001, 0.5)
        current_sps = np.clip(current_sps, 1, 20)
        model_sps_plus.fit(df)
        loss_sps_plus = np.mean(performance_metrics(cross_validation(
            model_sps_plus, initial='365.25 days', period='180 days', horizon='90 days'
        ))['rmse'])
        
        grad_sps = (loss_sps_plus - current_loss) / epsilon
        
        # Mise à jour des paramètres (descente de gradient)
        current_cps -= learning_rate * grad_cps
        current_sps -= learning_rate * grad_sps
        
        # Contraintes physiques
        current_cps = np.clip(current_cps, 0.001, 0.5)
        current_sps = np.clip(current_sps, 1, 20)
        
        history.append({
            'epoch': epoch,
            'cps': current_cps,
            'sps': current_sps,
            'loss': current_loss
        })
        
        print(f"Epoch {epoch}: RMSE={current_loss:.4f} | cps={current_cps:.4f} | sps={current_sps:.4f}")
    
    return best_params, history

# Application de Prophet

In [ ]:
# Création des graphiques et application de Prophet
for df, name in zip(dfs, df_names):
    # Extraction de la série temporelle
    df_prophet = df[['weighted_estimation_ges_month']].reset_index()
    df_prophet['month'] = df_prophet['month'].dt.to_timestamp('s').dt.strftime('%Y-%m-%d %H:%M:%S.000')
    df_prophet.columns = ['ds', 'y']  # Prophet attend 'ds' pour les dates et 'y' pour la variable à prédire
    print(df_prophet)
    print(df_prophet.dtypes)
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])    

    best_params, history = prophet_hyperparameter_tuning(
    df_prophet, 
    epochs=0, 
    learning_rate=0.1)

    plt.figure(figsize=(12, 6))
    plt.plot([h['epoch'] for h in history], [h['loss'] for h in history], label='RMSE')
    plt.xlabel('Epochs')
    plt.ylabel('Erreur (RMSE)')
    plt.title(f'Progression de la Descente de Gradient pour {name}')
    plt.legend()
    plt.grid()
    plt.show()

    # Créer un modèle Prophet
    model = Prophet()
    changepoint_prior_scale=best_params['changepoint_prior_scale'],
    seasonality_prior_scale=best_params['seasonality_prior_scale'])
    model.fit(df_prophet)

    with open(f'output {name}.txt', 'w') as f:
      print(f"Changepoint : {best_params['changepoint_prior_scale']}", file=f)
      print(f"Seasonality : {best_params['seasonality_prior_scale']}", file=f)

    # Faire des prédictions sur les données historiques
    historical_forecast = model.predict()

    # Faire des prédictions sur les données futures
    future = list()
    for j in range(2025, 2031):
        for i in range(1, 13):
            date = '%04d-%02d' % (j, i)
            future.append([date])
    future = pandas.DataFrame(future)
    future.columns = ['ds']
    future['ds']= pandas.to_datetime(future['ds'])
    future_forecast = model.predict(future)
    forecast = pd.concat([historical_forecast, future_forecast], axis=0)
    # Tracer les résultats
    plt.figure(figsize=(10, 6))
    
    plt.plot(df_prophet['ds'], df_prophet['y'], 
            label='Données réelles', 
            color='blue',
            marker='o',
            markersize=4,
            linewidth=1)

    # Tracer les prédictions centrales
    plt.plot(forecast['ds'], forecast['yhat'], 
            label='Prédiction centrale', 
            color='red', 
            linewidth=2)

    # Tracer l'enveloppe d'incertitude (intervalle de confiance à 80% par défaut)
    plt.fill_between(forecast['ds'], 
                    forecast['yhat_lower'], 
                    forecast['yhat_upper'],
                    color='orange', 
                    alpha=0.3,
                    label='Intervalle de confiance')

    # Ajouter une ligne verticale pour séparer historique/futur si vous avez des prédictions futures
    if 'future_forecast' in locals():
        last_historical_date = df_prophet['ds'].max()
        plt.axvline(x=last_historical_date, 
                    color='green', 
                    linestyle='--',
                    linewidth=1,
                    label='Début des prédictions')

    # Améliorations cosmétiques
    plt.title(f'Prédictions avec intervalles de confiance pour {name}', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Valeur prédite', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(loc='upper left', fontsize=10)
    plt.tight_layout()
    #plt.show()

    #filename = f"predictions_{name}.png"
    plt.savefig(f"predictions_{name}.png")
    
    # Plot des composantes de Prophet (tendance, saisonnalités)
    # Graphique de la tendance
    plt.figure(figsize=(10, 6))
    plt.plot(forecast['ds'], forecast['trend'], label='Tendance', color='green')
    plt.title(f'Tendance des données pour {name}')
    plt.xlabel('Date')
    plt.ylabel('Tendance')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    #plt.show()

    plt.savefig(f"components_{name}.png")
    
    # Tracer la saisonnalité annuelle (yearly seasonality)
    plt.figure(figsize=(10, 6))
    plt.plot(forecast['ds'], forecast['yearly'], label='Saisonnalité Annuelle', color='blue')
    plt.title(f'Saisonnalité Annuelle (Yearly Seasonality) pour {name}')
    plt.xlabel('Date')
    plt.ylabel('Saisonnalité Annuelle')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    #plt.show()

    #filename = f"trends_{name}.png"
    plt.savefig(f"trends_{name}.png")

# Calcul de R^2 et MSE

In [ ]:
for df, name in zip(dfs, df_names):
    # Lire les hyperparamètres sauvegardés
    with open(f'output {name}.txt', 'r') as f:
        content = f.read()
    cps = float(re.search(r"Changepoint : (\d+\.\d+)", content).group(1))
    sps = float(re.search(r"Seasonality : (\d+\.\d+)", content).group(1))

    # Extraction de la série temporelle
    df_prophet = df[['weighted_estimation_ges_month']].reset_index()
    df_prophet['month'] = df_prophet['month'].dt.to_timestamp('s').dt.strftime('%Y-%m-%d %H:%M:%S.000')
    df_prophet.columns = ['ds', 'y']  # Prophet attend 'ds' pour les dates et 'y' pour la variable à prédire
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

    # Prendre uniquement en compte les donneés dès 2013 (comme les calculs de R^2 et MSE sur les autres modèles)    
    df_prophet = df_prophet[df_prophet['ds'] >= datetime(2013, 1, 1)]


    # Lancement de Prophet
    model = Prophet(
        changepoint_prior_scale=cps,
        seasonality_prior_scale=sps
    )
    model.fit(df_prophet)
    forecast = model.predict(df_prophet)
    
    # Calcul d'erreur
    r2 = r2_score(df_prophet['y'], forecast['yhat'])
    mse = mean_squared_error(df_prophet['y'], forecast['yhat']) 

    print(f"Résultats pour {name}:")
    print(f"- Changepoint prior scale utilisé: {cps}")
    print(f"- Seasonality prior scale utilisé: {sps}")
    print(f"- R^2: {r2:.4f} (plus proche de 1 = meilleur)")
    print(f"- MSE: {mse:.4f} (plus proche de 0 = meilleur)")